# Support Vector Machines (SVM) - RBF Kernel

In [1]:
import sys
sys.path.insert(1, '../../libs')
from utils import get_data, temporal_train_test_split
from autorship import AuthorClassifier
from sklearn.preprocessing import MaxAbsScaler
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.tokenize import TweetTokenizer
import numpy as np
import pandas as pd

In [2]:
data = get_data("../../data/authors.csv")

In [3]:
tt = TweetTokenizer()

vectorizers = [CountVectorizer(ngram_range=(1,1), analyzer="word", tokenizer=tt.tokenize), 
                CountVectorizer(ngram_range=(1,3), analyzer="word", tokenizer=tt.tokenize), 
                CountVectorizer(ngram_range=(1,5), analyzer="char"),
                CountVectorizer(ngram_range=(4,5), analyzer="char"),
                CountVectorizer(ngram_range=(3,8), analyzer="char"),
                TfidfVectorizer(ngram_range=(1,1), analyzer="word", tokenizer=tt.tokenize), 
                TfidfVectorizer(ngram_range=(1,3), analyzer="word", tokenizer=tt.tokenize), 
                TfidfVectorizer(ngram_range=(1,5), analyzer="char"),
                TfidfVectorizer(ngram_range=(4,5), analyzer="char"),
                TfidfVectorizer(ngram_range=(3,8), analyzer="char")]

In [4]:
from joblib import Parallel, delayed
def process(vectorizer):
    evaluation = list()
    usernames = list(np.unique(data["username"]))
    
    vectorizer_str = vectorizer.__str__()
    print(f"Running => {vectorizer_str}")

    for i in range(len(usernames)):
        author1 = usernames.pop()

        for author2 in usernames:
            X_train, X_test, y_train, y_test = temporal_train_test_split(   
                data, author1, author2)

            clf = AuthorClassifier(vectorizer, clf=SVC(random_state=42), scaler=MaxAbsScaler())
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            evaluation.append(clf.evaluate(y_test, y_pred))
            metrics = pd.DataFrame(evaluation)[["f1_macro", "recall_macro", "precision_macro", "accuracy", "auc_score"]].mean()
    
    return (vectorizer_str, metrics)
    
results = Parallel(n_jobs=6)(delayed(process)(vectorizer) for vectorizer in vectorizers)
print(results) 

Running => CountVectorizer(tokenizer=<bound method TweetTokenizer.tokenize of <nltk.tokenize.casual.TweetTokenizer object at 0x7fe433ec9c70>>)
Running => CountVectorizer(ngram_range=(1, 3),
                tokenizer=<bound method TweetTokenizer.tokenize of <nltk.tokenize.casual.TweetTokenizer object at 0x7fd6391c7cd0>>)
Running => CountVectorizer(analyzer='char', ngram_range=(1, 5))
Running => CountVectorizer(analyzer='char', ngram_range=(3, 8))
Running => CountVectorizer(analyzer='char', ngram_range=(4, 5))
Running => TfidfVectorizer(tokenizer=<bound method TweetTokenizer.tokenize of <nltk.tokenize.casual.TweetTokenizer object at 0x7fd32948ecd0>>)
Running => TfidfVectorizer(ngram_range=(1, 3),
                tokenizer=<bound method TweetTokenizer.tokenize of <nltk.tokenize.casual.TweetTokenizer object at 0x7fe433ec9c10>>)
Running => TfidfVectorizer(analyzer='char', ngram_range=(1, 5))
Running => TfidfVectorizer(analyzer='char', ngram_range=(4, 5))
Running => TfidfVectorizer(analyzer=

In [5]:
metrics_df = pd.DataFrame([results[i][1] for i in range(len(results))])
metrics_df["vectorizer"] = [results[i][0] for i in range(len(results))]
metrics_df.to_csv("../../results/rbf_svm.csv")
metrics_df

,f1_macro,recall_macro,precision_macro,accuracy,auc_score,vectorizer
0,0.759409,0.765502,0.785721,0.763986,0.863289,CountVectorizer(tokenizer=<bound method TweetT...
1,0.591984,0.646559,0.757005,0.642910,0.849041,"CountVectorizer(ngram_range=(1, 3),\n ..."
2,0.746405,0.752573,0.771214,0.751043,0.844445,"CountVectorizer(analyzer='char', ngram_range=(..."
3,0.721582,0.730310,0.753527,0.728595,0.821004,"CountVectorizer(analyzer='char', ngram_range=(..."
4,0.670870,0.692621,0.741661,0.689961,0.813759,"CountVectorizer(analyzer='char', ngram_range=(..."
5,0.769654,0.774462,0.801371,0.776549,0.878803,TfidfVectorizer(tokenizer=<bound method TweetT...
6,0.685745,0.701625,0.749549,0.704631,0.817356,"TfidfVectorizer(ngram_range=(1, 3),\n ..."
7,0.587097,0.636559,0.737246,0.640849,0.784270,"TfidfVectorizer(analyzer='char', ngram_range=(..."
8,0.591876,0.635435,0.718572,0.639989,0.785779,"TfidfVectorizer(analyzer='char', ngram_range=(..."
9,0.540611,0.600653,0.695154,0.605711,0.766034,"TfidfVectorizer(analyzer='char', ngram_range=(..."


Row with max sum of columns (remember that by the result of f1-score we use the precision and recall score)

In [6]:
print(metrics_df.loc[metrics_df[['f1_macro', 'accuracy', 'auc_score']].sum(1).idxmax()])
print("\nVectorizer:", metrics_df.loc[metrics_df[['f1_macro', 'accuracy', 'auc_score']].sum(1).idxmax()]["vectorizer"])

f1_macro                                                    0.769654
recall_macro                                                0.774462
precision_macro                                             0.801371
accuracy                                                    0.776549
auc_score                                                   0.878803
vectorizer         TfidfVectorizer(tokenizer=<bound method TweetT...
Name: 5, dtype: object

Vectorizer: TfidfVectorizer(tokenizer=<bound method TweetTokenizer.tokenize of <nltk.tokenize.casual.TweetTokenizer object at 0x7fd32948ecd0>>)


In [7]:
pd.read_csv("../../results/rbf_svm.csv").drop("Unnamed: 0", axis=1)

,f1_macro,recall_macro,precision_macro,accuracy,auc_score,vectorizer
0,0.759409,0.765502,0.785721,0.763986,0.863289,CountVectorizer(tokenizer=<bound method TweetT...
1,0.591984,0.646559,0.757005,0.642910,0.849041,"CountVectorizer(ngram_range=(1, 3),\n ..."
2,0.746405,0.752573,0.771214,0.751043,0.844445,"CountVectorizer(analyzer='char', ngram_range=(..."
3,0.721582,0.730310,0.753527,0.728595,0.821004,"CountVectorizer(analyzer='char', ngram_range=(..."
4,0.670870,0.692621,0.741661,0.689961,0.813759,"CountVectorizer(analyzer='char', ngram_range=(..."
5,0.769654,0.774462,0.801371,0.776549,0.878803,TfidfVectorizer(tokenizer=<bound method TweetT...
6,0.685745,0.701625,0.749549,0.704631,0.817356,"TfidfVectorizer(ngram_range=(1, 3),\n ..."
7,0.587097,0.636559,0.737246,0.640849,0.784270,"TfidfVectorizer(analyzer='char', ngram_range=(..."
8,0.591876,0.635435,0.718572,0.639989,0.785779,"TfidfVectorizer(analyzer='char', ngram_range=(..."
9,0.540611,0.600653,0.695154,0.605711,0.766034,"TfidfVectorizer(analyzer='char', ngram_range=(..."
